# Projet : Extraction et Recommandation de films et séries avec rdflib et SPARQL

## Objectif
L’objectif de ce mini-projet est l’extraction des données de la base de données DBPedia en utilisant le langage de requêtes SPARQL afin d'alimenter la base de données de séries que vous avez créé au premier TP, l’analyse de ces données et la recommandation de séries et de films.

# Description

Vous allez créer un système de recommandation de séries/films basé sur des données RDF qui stockent des informations sur les films, les utilisateurs et leurs préférences cinématographiques.

Les étapes du projet sont comme suit :

## 1.  Extraction des données

a)  Vous utiliserez Rdflib pour accéder aux données et vous effectuerez des requêtes SPARQL pour les extraire. Vous êtes libres dans le choix et la taille des données que vous allez extraire. Le jeu de données doit néanmoins être représentatif pour pouvoir fournir des recommandations précises. 
    
   - Les données à extraire : 
       - Films : Chaque film a un titre, un réalisateur, une année de sortie, un genre (e.g. : action, comédie, science-fiction), un résumé, une liste d'acteurs principaux, une durée, une évaluation du film, etc.
       - Réalisateur : Chaque réalisateur a un nom, une biographie et une liste de films qu'il a réalisés.
       - Acteurs : Chaque acteur a un nom, une biographie et une liste de films dans lesquels il a joué.
       - Genres :  Chaque genre a un nom et une description.
       - Utilisateurs : chaque utilisateur a un identifiant et des préférences cinématographiques (acteurs préférés, genres préférés, etc).
       - Évaluations : Elle est décrite par l'identifiant de l'utilisateur qui a donné l'évaluation, identifiant du film évalué la note attribuée au film, Commentaire ou avis sur le film.

   - Liens entre les entités : 

        - Les films sont associés à leurs acteurs, réalisateurs et genres.
        - Les utilisateurs sont associés aux films qu'ils ont évalués.
        - Les utilisateurs peuvent être liés entre eux en fonction de leurs préférences cinématographiques similaires.

   b)  Transformer les données en triplets RDF :  Vous allez transformer ces résultats en triplets RDF avant de les ajouter à votre graphe RDF existant. 
  
  c) Ajouter les données au graphe existant : Utilisez la méthode g.add() de votre graphe RDF, que vous avez créé au premier TP, pour ajouter les triplets RDF représentant les données DBpedia que vous avez transformées.

In [12]:
from rdflib import Graph, Namespace, URIRef, Literal
from rdflib.namespace import RDF, FOAF, XSD

# Créer un graphe RDF
rdf_graph = Graph()

# Définir les namespaces
EX = Namespace("http://example.org/")
DBO = Namespace("http://dbpedia.org/ontology/")
rdf_graph.bind("ex", EX)
rdf_graph.bind("dbo", DBO)
rdf_graph.bind("foaf", FOAF)

# Fonction pour ajouter un triplet RDF
def add_triplet(graph, subj, pred, obj):
    if isinstance(obj, str) and obj.startswith("http"):
        obj = URIRef(obj)  # Convertir les URIs
    elif isinstance(obj, (str, int, float)):
        obj = Literal(obj)  # Convertir en littéral
    graph.add((subj, pred, obj))

# Transformer les résultats JSON en triplets RDF
def transform_results_to_rdf(results, rdf_graph, entity_type, properties_map):
    for result in results["results"]["bindings"]:
        # Assurez-vous que les valeurs des URIs sont bien des URIRef
        entity_uri = result.get(entity_type, {}).get("value")
        if not entity_uri:
            continue

        entity_uri = URIRef(entity_uri)  # Convertir en URIRef si nécessaire

        for prop_name, prop_info in properties_map.items():
            prop_uri = prop_info["uri"]
            prop_value = result.get(prop_info["value_key"], {}).get("value", "N/A")

            # Convertir les valeurs en Literal ou URIRef selon le cas
            if prop_value != "N/A":
                if prop_info.get("is_entity_link"):
                    # Si c'est une entité liée, convertir en URIRef
                    rdf_graph.add((entity_uri, URIRef(prop_uri), URIRef(prop_value)))
                else:
                    # Sinon, c'est un Literal
                    rdf_graph.add((entity_uri, URIRef(prop_uri), Literal(prop_value)))
            else:
                # Si la valeur est manquante, ajoutez un Literal vide ou une valeur par défaut
                rdf_graph.add((entity_uri, URIRef(prop_uri), Literal("N/A")))

    return rdf_graph

def clean_uri(uri):
    """ Nettoie et prépare une URI en supprimant les espaces ou les caractères inutiles. """
    return uri.strip().replace(" ", "_")



In [13]:
# Mapping des propriétés pour les films
film_properties_map = {
    "title": {
        "uri": "http://www.w3.org/2000/01/rdf-schema#label",
        "value_key": "title"
    },
    "director": {
        "uri": "http://dbpedia.org/ontology/director",
        "value_key": "director",
        "is_entity_link": True  # L'entité liée est le réalisateur
    },
    "release_date": {
        "uri": "http://dbpedia.org/ontology/releaseDate",
        "value_key": "releaseYear"
    },
    "genre": {
        "uri": "http://dbpedia.org/ontology/genre",
        "value_key": "genre"
    },
    "summary": {
        "uri": "http://dbpedia.org/ontology/abstract",
        "value_key": "summary"
    },
    "actors": {
        "uri": "http://dbpedia.org/ontology/starring",
        "value_key": "actorsList",
        "is_entity_link": True  # Liste d'entités liées (acteurs)
    },
    "duration": {
        "uri": "http://dbpedia.org/ontology/runtime",
        "value_key": "duration"
    },
    "rating": {
        "uri": "http://dbpedia.org/ontology/rating",
        "value_key": "rating"
    }
}

# Mapping des propriétés pour les réalisateurs
director_properties_map = {
    "name": {
        "uri": "http://www.w3.org/2000/01/rdf-schema#label",
        "value_key": "name"
    },
    "bio": {
        "uri": "http://dbpedia.org/ontology/abstract",
        "value_key": "bio"
    },
    "films": {
        "uri": "http://dbpedia.org/ontology/film",
        "value_key": "films",
        "is_entity_link": True  # Liste de films réalisés par ce réalisateur
    }
}

# Mapping des propriétés pour les acteurs
actor_properties_map = {
    "name": {
        "uri": "http://www.w3.org/2000/01/rdf-schema#label",
        "value_key": "name"
    },
    "bio": {
        "uri": "http://dbpedia.org/ontology/abstract",
        "value_key": "bio"
    },
    "films": {
        "uri": "http://dbpedia.org/ontology/film",
        "value_key": "films",
        "is_entity_link": True  # Liste de films dans lesquels l'acteur a joué
    }
}

# Mapping des propriétés pour les genres
genre_properties_map = {
    "name": {
        "uri": "http://www.w3.org/2000/01/rdf-schema#label",
        "value_key": "name"
    },
    "description": {
        "uri": "http://dbpedia.org/ontology/abstract",
        "value_key": "description"
    },
    "films": {
        "uri": "http://dbpedia.org/ontology/film",
        "value_key": "films",
        "is_entity_link": True  # Liste de films correspondant à ce genre
    }
}

# Mapping des propriétés pour les utilisateurs
user_properties_map = {
    "id": {
        "uri": "http://dbpedia.org/ontology/identifier",
        "value_key": "id"
    },
    "name": {
        "uri": "http://dbpedia.org/ontology/name",
        "value_key": "name"
    },
    "favorite_actors": {
        "uri": "http://dbpedia.org/ontology/favoriteActor",
        "value_key": "favoriteActors",
        "is_entity_link": True  # Liste des acteurs préférés de l'utilisateur
    },
    "favorite_genres": {
        "uri": "http://dbpedia.org/ontology/favoriteGenre",
        "value_key": "favoriteGenres",
        "is_entity_link": True  # Liste des genres préférés de l'utilisateur
    }
}

# Mapping des propriétés pour les évaluations
rating_properties_map = {
    "user_id": {
        "uri": "http://dbpedia.org/ontology/userId",
        "value_key": "userId"
    },
    "film_id": {
        "uri": "http://dbpedia.org/ontology/filmId",
        "value_key": "filmId"
    },
    "rating_value": {
        "uri": "http://dbpedia.org/ontology/ratingValue",
        "value_key": "ratingValue"
    },
    "comment": {
        "uri": "http://dbpedia.org/ontology/comment",
        "value_key": "comment"
    }
}


In [14]:
from SPARQLWrapper import SPARQLWrapper, JSON
import urllib.parse

# URL du point d'accès SparQL de la DBpedia
dbpedia_sparql_endpoint = "http://dbpedia.org/sparql"

# Création d'une instance de SPARQLWrapper
sparql = SPARQLWrapper(dbpedia_sparql_endpoint)

# Requête SPARQL mise à jour
query = """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX dbo: <http://dbpedia.org/ontology/>

SELECT ?film ?title ?director ?releaseYear ?genre ?summary ?duration ?rating (GROUP_CONCAT(?actor; SEPARATOR=", ") AS ?actorsList)
WHERE {
    ?film rdf:type dbo:Film .                 # Les entités de type Film
    ?film rdfs:label ?title .                # Titre du film
    ?film dbo:director ?director .           # Réalisateur
    ?film dbo:releaseDate ?releaseYear .     # Année de sortie
    ?film dbo:genre ?genre .                 # Genre
    ?film dbo:abstract ?summary .            # Résumé
    ?film dbo:starring ?actor .              # Acteurs (URI des acteurs)
    ?film dbo:runtime ?duration .            # Durée
    OPTIONAL { ?film dbo:rating ?rating }    # Évaluation (optionnelle)
    FILTER (lang(?title) = "en")             # Filtrer les titres en anglais
    FILTER (lang(?summary) = "en")           # Filtrer les résumés en anglais
}
GROUP BY ?film ?title ?director ?releaseYear ?genre ?summary ?duration ?rating
LIMIT 100
"""

# Format de résultat en JSON
sparql.setReturnFormat(JSON)

# Exécution de la requête
sparql.setQuery(query)
results = sparql.query().convert()

# Affichage des résultats
for result in results["results"]["bindings"]:
    film = result.get("film", {}).get("value", "N/A")
    title = result.get("title", {}).get("value", "N/A")
    director = result.get("director", {}).get("value", "N/A")
    releaseYear = result.get("releaseYear", {}).get("value", "N/A")
    genre = result.get("genre", {}).get("value", "N/A")
    summary = result.get("summary", {}).get("value", "N/A")
    actorsList = result.get("actorsList", {}).get("value", "N/A")  # Liste des URI des acteurs
    duration = result.get("duration", {}).get("value", "N/A")
    rating = result.get("rating", {}).get("value", "N/A")

    # Nettoyage des URI des acteurs (si nécessaire)
    if actorsList != "N/A":
        actorsList = [clean_uri(actor.strip()) for actor in actorsList.split(",")]

    '''#print(f"Film: {film}")
    print(f"  Title: {title}")
    #print(f"  Director: {director}")
    #print(f"  Release Year: {releaseYear}")
    print(f"  Genre: {genre}")
    print(f"  Summary: {summary}")
    print(f"  Actors: {', '.join(actorsList)}")
    print(f"  Duration: {duration}")
    print(f"  Rating: {rating}\n")'''

# Ajouter les résultats des films au graphe RDF
rdf_graph = transform_results_to_rdf(results, rdf_graph, "film", film_properties_map)


http://dbpedia.org/resource/Mary_Page_Keller, http://dbpedia.org/resource/Meghann_Haldeman, http://dbpedia.org/resource/Tina_Majorino, http://dbpedia.org/resource/Hilary_Swank, http://dbpedia.org/resource/Jay_Mohr, http://dbpedia.org/resource/Jerry_O'Connell, http://dbpedia.org/resource/Margaret_Langrick does not look like a valid URI, trying to serialize this will break.
http://dbpedia.org/resource/Vernon_Wells_(actor), http://dbpedia.org/resource/Edward_Albert, http://dbpedia.org/resource/Michael_Copon, http://dbpedia.org/resource/Erin_Cahill, http://dbpedia.org/resource/Jason_Faunt does not look like a valid URI, trying to serialize this will break.
http://dbpedia.org/resource/Vernon_Wells_(actor), http://dbpedia.org/resource/Edward_Albert, http://dbpedia.org/resource/Michael_Copon, http://dbpedia.org/resource/Erin_Cahill, http://dbpedia.org/resource/Jason_Faunt does not look like a valid URI, trying to serialize this will break.
http://dbpedia.org/resource/Vernon_Wells_(actor), htt

Film version 1

In [3]:
from SPARQLWrapper import SPARQLWrapper, JSON

# URL du point d'accès SparQL de la DBpedia
dbpedia_sparql_endpoint = "http://dbpedia.org/sparql"

# Création d'une instance de SPARQLWrapper
sparql = SPARQLWrapper(dbpedia_sparql_endpoint)

# Requête SPARQL mise à jour
query = """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX dbo: <http://dbpedia.org/ontology/>

SELECT ?film ?title ?director ?releaseYear ?genre ?summary ?duration ?rating (GROUP_CONCAT(?actor; SEPARATOR=", ") AS ?actorsList)
WHERE {
    ?film rdf:type dbo:Film .                 # Les entités de type Film
    ?film rdfs:label ?title .                # Titre du film
    ?film dbo:director ?director .           # Réalisateur
    ?film dbo:releaseDate ?releaseYear .     # Année de sortie
    ?film dbo:genre ?genre .                 # Genre
    ?film dbo:abstract ?summary .            # Résumé
    ?film dbo:starring ?actor .              # Acteurs (URI des acteurs)
    ?film dbo:runtime ?duration .            # Durée
    OPTIONAL { ?film dbo:rating ?rating }    # Évaluation (optionnelle)
    FILTER (lang(?title) = "en")             # Filtrer les titres en anglais
    FILTER (lang(?summary) = "en")           # Filtrer les résumés en anglais
}
GROUP BY ?film ?title ?director ?releaseYear ?genre ?summary ?duration ?rating
LIMIT 100
"""

# Format de résultat en JSON
sparql.setReturnFormat(JSON)

# Exécution de la requête
sparql.setQuery(query)
results = sparql.query().convert()

# Affichage des résultats
for result in results["results"]["bindings"]:
    film = result.get("film", {}).get("value", "N/A")
    title = result.get("title", {}).get("value", "N/A")
    director = result.get("director", {}).get("value", "N/A")
    releaseYear = result.get("releaseYear", {}).get("value", "N/A")
    genre = result.get("genre", {}).get("value", "N/A")
    summary = result.get("summary", {}).get("value", "N/A")
    actorsList = result.get("actorsList", {}).get("value", "N/A")  # Liste des URI des acteurs
    duration = result.get("duration", {}).get("value", "N/A")
    rating = result.get("rating", {}).get("value", "N/A")

    print(f"Film: {film}")
    print(f"  Title: {title}")
    print(f"  Director: {director}")
    print(f"  Release Year: {releaseYear}")
    print(f"  Genre: {genre}")
    print(f"  Summary: {summary}")
    print(f"  Actors: {actorsList}")
    print(f"  Duration: {duration}")
    print(f"  Rating: {rating}\n")
# Ajouter les résultats des films au graphe RDF
rdf_graph = transform_results_to_rdf(results, rdf_graph, "film")

http://dbpedia.org/resource/Mary_Page_Keller, http://dbpedia.org/resource/Meghann_Haldeman, http://dbpedia.org/resource/Tina_Majorino, http://dbpedia.org/resource/Hilary_Swank, http://dbpedia.org/resource/Jay_Mohr, http://dbpedia.org/resource/Jerry_O'Connell, http://dbpedia.org/resource/Margaret_Langrick does not look like a valid URI, trying to serialize this will break.
http://dbpedia.org/resource/Vernon_Wells_(actor), http://dbpedia.org/resource/Edward_Albert, http://dbpedia.org/resource/Michael_Copon, http://dbpedia.org/resource/Erin_Cahill, http://dbpedia.org/resource/Jason_Faunt does not look like a valid URI, trying to serialize this will break.
http://dbpedia.org/resource/Vernon_Wells_(actor), http://dbpedia.org/resource/Edward_Albert, http://dbpedia.org/resource/Michael_Copon, http://dbpedia.org/resource/Erin_Cahill, http://dbpedia.org/resource/Jason_Faunt does not look like a valid URI, trying to serialize this will break.
http://dbpedia.org/resource/Vernon_Wells_(actor), htt

Film: http://dbpedia.org/resource/Camp_Wilder
  Title: Camp Wilder
  Director: http://dbpedia.org/resource/Arlene_Sanford
  Release Year: 1992-09-18
  Genre: http://dbpedia.org/resource/Sitcom
  Summary: Camp Wilder is an American television sitcom which aired on ABC from September 18, 1992 until February 26, 1993. The premise centered on a young woman who opens up her home to the friends of her younger siblings, who sought it as judgment-free "hangout", and who regularly went to her for advice. The series was created by Matthew Carlson, and produced by a.k.a. Productions in association with Capital Cities Entertainment. The show aired as a part of ABC's TGIF lineup, but was cancelled after 19 episodes due to low ratings. A twentieth episode was produced, but was never aired in the United States. Camp Wilder was also shown in the UK, Spain and Germany.
  Actors: http://dbpedia.org/resource/Mary_Page_Keller, http://dbpedia.org/resource/Meghann_Haldeman, http://dbpedia.org/resource/Tina_

Réalisateur V1

In [15]:
# Requête SPARQL
query = """
PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX foaf: <http://xmlns.com/foaf/0.1/>
PREFIX dbr: <http://dbpedia.org/resource/>

SELECT ?director ?name ?bio (GROUP_CONCAT(?film; SEPARATOR=", ") AS ?films)
WHERE {
    ?director a dbo:Person .                 # L'entité est une personne
    ?director dbo:occupation dbr:Film_director . # La personne est un réalisateur
    ?director foaf:name ?name .             # Nom du réalisateur
    ?director dbo:abstract ?bio .           # Biographie
    ?film dbo:director ?director .          # Film réalisé par cette personne
    FILTER (lang(?bio) = "en")              # Filtrer les biographies en anglais
}
GROUP BY ?director ?name ?bio
LIMIT 50
"""

# Format de résultat en JSON
sparql.setReturnFormat(JSON)

# Exécution de la requête
sparql.setQuery(query)
results = sparql.query().convert()

# Affichage des résultats
for result in results["results"]["bindings"]:
    director = result.get("director", {}).get("value", "N/A")
    name = result.get("name", {}).get("value", "N/A")
    bio = result.get("bio", {}).get("value", "N/A")
    films = result.get("films", {}).get("value", "N/A")  # Liste des films sous forme d'URI


    '''print(f"Director URI: {director}")
    print(f"  Name: {name}")
    print(f"  Bio: {bio}")
    print(f"  Films (URIs): {films}\n")'''
rdf_graph = transform_results_to_rdf(results, rdf_graph, "director", director_properties_map)



http://dbpedia.org/resource/Say_Anything..., http://dbpedia.org/resource/Pearl_Jam_Twenty, http://dbpedia.org/resource/Vanilla_Sky, http://dbpedia.org/resource/Elizabethtown_(film), http://dbpedia.org/resource/The_Union_(2011_film), http://dbpedia.org/resource/Jerry_Maguire__Jerry_Maguire__1, http://dbpedia.org/resource/Singles_(1992_film), http://dbpedia.org/resource/We_Bought_a_Zoo, http://dbpedia.org/resource/Almost_Famous, http://dbpedia.org/resource/Aloha_(2015_film) does not look like a valid URI, trying to serialize this will break.
http://dbpedia.org/resource/Be_Seeing_You,_Father, http://dbpedia.org/resource/Queen_of_the_Scala, http://dbpedia.org/resource/Totòtruffa_62, http://dbpedia.org/resource/Gli_eroi_del_doppio_gioco, http://dbpedia.org/resource/The_Last_Dance_(1941_film), http://dbpedia.org/resource/The_Most_Beautiful_Couple_in_the_World, http://dbpedia.org/resource/La_cambiale, http://dbpedia.org/resource/Anna's_Sin, http://dbpedia.org/resource/Lost_in_the_Dark_(1947_f

In [17]:
from SPARQLWrapper import SPARQLWrapper, JSON
import urllib.parse
from rdflib import Graph, URIRef, Literal

# Fonction pour nettoyer et échapper les URI
def clean_uri(uri):
    return urllib.parse.quote(uri, safe=":/?&=[];#")

# URL du point d'accès SparQL de la DBpedia
dbpedia_sparql_endpoint = "http://dbpedia.org/sparql"

# Création d'une instance de SPARQLWrapper
sparql = SPARQLWrapper(dbpedia_sparql_endpoint)

# Requête SPARQL mise à jour
query = """
PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX foaf: <http://xmlns.com/foaf/0.1/>
PREFIX dbr: <http://dbpedia.org/resource/>

SELECT ?director ?name ?bio (GROUP_CONCAT(?film; SEPARATOR=", ") AS ?films)
WHERE {
    ?director a dbo:Person .                 # L'entité est une personne
    ?director dbo:occupation dbr:Film_director . # La personne est un réalisateur
    ?director foaf:name ?name .             # Nom du réalisateur
    ?director dbo:abstract ?bio .           # Biographie
    ?film dbo:director ?director .          # Film réalisé par cette personne
    FILTER (lang(?bio) = "en")              # Filtrer les biographies en anglais
}
GROUP BY ?director ?name ?bio
LIMIT 50
"""

# Format de résultat en JSON
sparql.setReturnFormat(JSON)

# Exécution de la requête
sparql.setQuery(query)
results = sparql.query().convert()

# Affichage des résultats et ajout au graphe RDF
for result in results["results"]["bindings"]:
    director = result.get("director", {}).get("value", "N/A")
    name = result.get("name", {}).get("value", "N/A")
    bio = result.get("bio", {}).get("value", "N/A")
    films = result.get("films", {}).get("value", "N/A")  # Liste des films sous forme d'URI

    # Nettoyage des URI des films (si nécessaire)
    if films != "N/A":
        films = [clean_uri(film.strip()) for film in films.split(",")]

    # Affichage des résultats
    '''print(f"Director URI: {director}")
    print(f"  Name: {name}")
    print(f"  Bio: {bio}")
    print(f"  Films (URIs): {', '.join(films)}\n")'''

# Ajouter les résultats des réalisateurs au même graphe RDF existant
rdf_graph = transform_results_to_rdf(results, rdf_graph, "director", director_properties_map)



http://dbpedia.org/resource/Say_Anything..., http://dbpedia.org/resource/Pearl_Jam_Twenty, http://dbpedia.org/resource/Vanilla_Sky, http://dbpedia.org/resource/Elizabethtown_(film), http://dbpedia.org/resource/The_Union_(2011_film), http://dbpedia.org/resource/Jerry_Maguire__Jerry_Maguire__1, http://dbpedia.org/resource/Singles_(1992_film), http://dbpedia.org/resource/We_Bought_a_Zoo, http://dbpedia.org/resource/Almost_Famous, http://dbpedia.org/resource/Aloha_(2015_film) does not look like a valid URI, trying to serialize this will break.
http://dbpedia.org/resource/Be_Seeing_You,_Father, http://dbpedia.org/resource/Queen_of_the_Scala, http://dbpedia.org/resource/Totòtruffa_62, http://dbpedia.org/resource/Gli_eroi_del_doppio_gioco, http://dbpedia.org/resource/The_Last_Dance_(1941_film), http://dbpedia.org/resource/The_Most_Beautiful_Couple_in_the_World, http://dbpedia.org/resource/La_cambiale, http://dbpedia.org/resource/Anna's_Sin, http://dbpedia.org/resource/Lost_in_the_Dark_(1947_f

RDF Graph (en Turtle) :


PluginException: No plugin registered for (rdf, <class 'rdflib.serializer.Serializer'>)

In [26]:
print("nb :", len(rdf_graph) )
for s, p, o in rdf_graph:
    print(s,', ',p,', ', o)

nb : 969
http://dbpedia.org/resource/The_Kingdom_(miniseries) ,  http://dbpedia.org/ontology/starring ,  http://dbpedia.org/resource/Birgitte_Raaberg, http://dbpedia.org/resource/Holger_Juul_Hansen, http://dbpedia.org/resource/Peter_Mygind, http://dbpedia.org/resource/Udo_Kier, http://dbpedia.org/resource/Ghita_Nørby, http://dbpedia.org/resource/Annevig_Schelde_Ebbe, http://dbpedia.org/resource/John_Hahn-Petersen, http://dbpedia.org/resource/Erik_Wedersøe, http://dbpedia.org/resource/Ernst-Hugo_Järegård, http://dbpedia.org/resource/Otto_Brandenburg, http://dbpedia.org/resource/Baard_Owe, http://dbpedia.org/resource/Solbjørg_Højfeldt, http://dbpedia.org/resource/Jens_Okking, http://dbpedia.org/resource/Birthe_Neumann, http://dbpedia.org/resource/Søren_Pilmark, http://dbpedia.org/resource/Kirsten_Rolffes, http://dbpedia.org/resource/Vita_Jensen
http://dbpedia.org/resource/Lizzie_Borden_Took_an_Ax ,  http://dbpedia.org/ontology/runtime ,  5220.0
http://dbpedia.org/resource/Power_Rangers_L

Acteurs

In [22]:
# Requête SPARQL
query = """
PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX foaf: <http://xmlns.com/foaf/0.1/>

SELECT ?actor ?name ?bio (GROUP_CONCAT(?film; SEPARATOR=", ") AS ?films)
WHERE {
    ?actor a dbo:Actor .                    # L'entité est un acteur
    ?actor foaf:name ?name .                # Nom de l'acteur
    ?actor dbo:abstract ?bio .              # Biographie
    ?film dbo:starring ?actor .             # Films dans lesquels l'acteur a joué
    FILTER (lang(?bio) = "en")              # Filtrer les biographies en anglais
}
GROUP BY ?actor ?name ?bio
LIMIT 50
"""

# Format de résultat en JSON
sparql.setReturnFormat(JSON)

# Exécution de la requête
sparql.setQuery(query)
results = sparql.query().convert()

# Affichage des résultats
for result in results["results"]["bindings"]:
    actor = result.get("actor", {}).get("value", "N/A")
    name = result.get("name", {}).get("value", "N/A")
    bio = result.get("bio", {}).get("value", "N/A")
    films = result.get("films", {}).get("value", "N/A")  # Liste des films sous forme d'URI

    # Nettoyage des URI des films (si nécessaire)
    if films != "N/A":
        films = [clean_uri(film.strip()) for film in films.split(",")]

    '''print(f"Actor URI: {actor}")
    print(f"  Name: {name}")
    print(f"  Bio: {bio}")
    print(f"  Films: {films}\n")'''

rdf_graph = transform_results_to_rdf(results, rdf_graph, "actor", actor_properties_map)



http://dbpedia.org/resource/The_Trouble_with_Angels_(film), http://dbpedia.org/resource/Mackenna's_Gold, http://dbpedia.org/resource/Downhill_Racer, http://dbpedia.org/resource/Nobody_Runs_Forever, http://dbpedia.org/resource/The_Greek_Tycoon, http://dbpedia.org/resource/Assignment_K, http://dbpedia.org/resource/Survival_Zone does not look like a valid URI, trying to serialize this will break.
http://dbpedia.org/resource/Hot_&_Saucy_Pizza_Girls, http://dbpedia.org/resource/The_Tale_of_Tiffany_Lust does not look like a valid URI, trying to serialize this will break.
http://dbpedia.org/resource/The_Sense_of_Wonder, http://dbpedia.org/resource/Curiosa_(film), http://dbpedia.org/resource/I_Wish_Someone_Were_Waiting_for_Me_Somewhere_(film) does not look like a valid URI, trying to serialize this will break.
http://dbpedia.org/resource/Deutsch-les-Landes, http://dbpedia.org/resource/Paula_(2016_film), http://dbpedia.org/resource/Riviera_(TV_series), http://dbpedia.org/resource/17_Girls, http

Genres

In [25]:
# Requête SPARQL
query = """
PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX foaf: <http://xmlns.com/foaf/0.1/>

SELECT ?genre ?name ?description
WHERE {
    ?genre a dbo:Genre .                     # L'entité est un genre
    ?genre foaf:name ?name .                 # Nom du genre
    ?genre dbo:abstract ?description .       # Description du genre
    FILTER (lang(?description) = "en")       # Filtrer les descriptions en anglais
}
LIMIT 20
"""

# Format de résultat en JSON
sparql.setReturnFormat(JSON)

# Exécution de la requête
sparql.setQuery(query)
results = sparql.query().convert()

# Affichage des résultats
for result in results["results"]["bindings"]:
    genre = result.get("genre", {}).get("value", "N/A")
    name = result.get("name", {}).get("value", "N/A")
    description = result.get("description", {}).get("value", "N/A")

    '''print(f"Genre URI: {genre}")
    print(f"  Name: {name}")
    print(f"  Description: {description}\n")'''
rdf_graph = transform_results_to_rdf(results, rdf_graph, "genre", genre_properties_map)


Utilisateur

In [29]:
# Requête SPARQL pour simuler des préférences utilisateurs
query = """
PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX foaf: <http://xmlns.com/foaf/0.1/>
PREFIX ex: <http://example.org/>

SELECT ?user ?userId ?preferredActor ?actorName ?preferredGenre ?genreName
WHERE {
    # Simulation d'utilisateurs fictifs
    ?user a ex:User .                        # L'entité utilisateur
    ?user ex:userId ?userId .                # Identifiant de l'utilisateur
    ?user ex:preferredActor ?preferredActor .# Acteur préféré
    ?preferredActor foaf:name ?actorName .   # Nom de l'acteur préféré
    ?user ex:preferredGenre ?preferredGenre .# Genre préféré
    ?preferredGenre foaf:name ?genreName .   # Nom du genre préféré

    # Exemples d'entités d'acteurs et de genres dans DBpedia
    ?preferredActor a dbo:Actor .            # Acteurs DBpedia
    ?preferredGenre a dbo:Genre .            # Genres DBpedia
}
LIMIT 20
"""

# Format de résultat en JSON
sparql.setReturnFormat(JSON)

# Exécution de la requête
sparql.setQuery(query)
results = sparql.query().convert()

# Affichage des résultats
for result in results["results"]["bindings"]:
    user = result.get("user", {}).get("value", "N/A")
    userId = result.get("userId", {}).get("value", "N/A")
    preferredActor = result.get("preferredActor", {}).get("value", "N/A")
    actorName = result.get("actorName", {}).get("value", "N/A")
    preferredGenre = result.get("preferredGenre", {}).get("value", "N/A")
    genreName = result.get("genreName", {}).get("value", "N/A")

    '''print(f"User URI: {user}")
    print(f"  User ID: {userId}")
    print(f"  Preferred Actor: {actorName} ({preferredActor})")
    print(f"  Preferred Genre: {genreName} ({preferredGenre})\n")'''

rdf_graph = transform_results_to_rdf(results, rdf_graph, "user", user_properties_map)

In [32]:
print("nb :", len(rdf_graph) )

nb : 969


Evaluations

In [31]:
# Requête SPARQL pour récupérer les évaluations
query = """
PREFIX ex: <http://example.org/>
PREFIX foaf: <http://xmlns.com/foaf/0.1/>
PREFIX dbo: <http://dbpedia.org/ontology/>

SELECT ?user ?userId ?film ?filmTitle ?rating ?comment
WHERE {
    ?evaluation a ex:Evaluation .            # Une entité de type Evaluation
    ?evaluation ex:user ?user .              # Utilisateur qui a donné l'évaluation
    ?user ex:userId ?userId .                # Identifiant de l'utilisateur
    ?evaluation ex:film ?film .              # Film évalué
    ?film foaf:name ?filmTitle .             # Titre du film
    ?evaluation ex:rating ?rating .          # Note attribuée au film
    ?evaluation ex:comment ?comment .        # Commentaire de l'utilisateur
}
LIMIT 20
"""

# Format de résultat en JSON
sparql.setReturnFormat(JSON)

# Exécution de la requête
sparql.setQuery(query)
results = sparql.query().convert()

# Affichage des résultats
for result in results["results"]["bindings"]:
    user = result.get("user", {}).get("value", "N/A")
    userId = result.get("userId", {}).get("value", "N/A")
    film = result.get("film", {}).get("value", "N/A")
    filmTitle = result.get("filmTitle", {}).get("value", "N/A")
    rating = result.get("rating", {}).get("value", "N/A")
    comment = result.get("comment", {}).get("value", "N/A")

    '''print(f"User: {user} (ID: {userId})")
    print(f"  Film: {filmTitle} ({film})")
    print(f"  Rating: {rating}")
    print(f"  Comment: {comment}\n")'''

rdf_graph = transform_results_to_rdf(results, rdf_graph, "rating", rating_properties_map)



## 2. Prétraitement des données   

Nettoyez et traitez les données extraites pour supprimer les doublons, gérer les valeurs manquantes et normaliser.

Prétraitement des données du graphe rdf_graph : Nettoyage

In [36]:
from rdflib import Graph, URIRef, Literal

def preprocess_rdf_graph(graph, default_values=None):
    default_values = default_values or {}
    cleaned_graph = Graph()

    for subject, predicate, obj in graph:
        # Vérification des URI (sujets et prédicats)
        if isinstance(subject, URIRef):
            subject = URIRef(subject.strip())
        if isinstance(predicate, URIRef):
            predicate = URIRef(predicate.strip())

        # Vérification des littéraux (objets)
        if isinstance(obj, Literal):
            obj = Literal(str(obj).strip())

        # Vérification des URI objets
        elif isinstance(obj, URIRef):
            obj = URIRef(obj.strip())

        # Complétion des valeurs manquantes
        if not obj and predicate in default_values:
            obj = Literal(default_values[predicate])

        # Ajouter uniquement des triplets valides
        if subject and predicate and obj:
            cleaned_graph.add((subject, predicate, obj))

    return cleaned_graph


In [39]:
default_values = {
    # Films
    URIRef("http://dbpedia.org/ontology/title"): "Untitled",  # Titre du film
    URIRef("http://dbpedia.org/ontology/director"): "Unknown director",  # Réalisateur
    URIRef("http://dbpedia.org/ontology/releaseDate"): "Unknown release date",  # Date de sortie
    URIRef("http://dbpedia.org/ontology/genre"): "Unspecified genre",  # Genre
    URIRef("http://dbpedia.org/ontology/abstract"): "No summary available",  # Résumé
    URIRef("http://dbpedia.org/ontology/starring"): "No main actors listed",  # Acteurs principaux
    URIRef("http://dbpedia.org/ontology/duration"): "Unknown duration",  # Durée
    URIRef("http://dbpedia.org/ontology/rating"): "Not rated",  # Évaluation

    # Réalisateurs
    URIRef("http://dbpedia.org/ontology/bio"): "Biography not available",  # Biographie
    URIRef("http://dbpedia.org/ontology/films"): "No films listed",  # Films réalisés

    # Acteurs
    URIRef("http://dbpedia.org/ontology/actorName"): "Unknown actor",  # Nom de l'acteur
    URIRef("http://dbpedia.org/ontology/biography"): "Biography not available",  # Biographie de l'acteur
    URIRef("http://dbpedia.org/ontology/filmography"): "No films listed",  # Filmographie

    # Genres
    URIRef("http://dbpedia.org/ontology/genreName"): "Unnamed genre",  # Nom du genre
    URIRef("http://dbpedia.org/ontology/description"): "No description available",  # Description du genre

    # Utilisateurs
    URIRef("http://dbpedia.org/ontology/userId"): "Unknown user",  # Identifiant utilisateur
    URIRef("http://dbpedia.org/ontology/preferences"): "No preferences listed",  # Préférences cinématographiques

    # Évaluations
    URIRef("http://dbpedia.org/ontology/reviewer"): "Unknown reviewer",  # Auteur de l'évaluation
    URIRef("http://dbpedia.org/ontology/reviewedFilm"): "Unknown film",  # Film évalué
    URIRef("http://dbpedia.org/ontology/ratingValue"): "No rating given",  # Note attribuée
    URIRef("http://dbpedia.org/ontology/reviewComment"): "No comment provided",  # Commentaire ou avis
}

rdf_graph = preprocess_rdf_graph(rdf_graph, default_values)
print("nb :", len(rdf_graph) )
for s, p, o in rdf_graph:
    print(f"Subject: {s}, Predicate: {p}, Object: {o}")


http://dbpedia.org/resource/Birgitte_Raaberg, http://dbpedia.org/resource/Holger_Juul_Hansen, http://dbpedia.org/resource/Peter_Mygind, http://dbpedia.org/resource/Udo_Kier, http://dbpedia.org/resource/Ghita_Nørby, http://dbpedia.org/resource/Annevig_Schelde_Ebbe, http://dbpedia.org/resource/John_Hahn-Petersen, http://dbpedia.org/resource/Erik_Wedersøe, http://dbpedia.org/resource/Ernst-Hugo_Järegård, http://dbpedia.org/resource/Otto_Brandenburg, http://dbpedia.org/resource/Baard_Owe, http://dbpedia.org/resource/Solbjørg_Højfeldt, http://dbpedia.org/resource/Jens_Okking, http://dbpedia.org/resource/Birthe_Neumann, http://dbpedia.org/resource/Søren_Pilmark, http://dbpedia.org/resource/Kirsten_Rolffes, http://dbpedia.org/resource/Vita_Jensen does not look like a valid URI, trying to serialize this will break.
http://dbpedia.org/resource/Ben_Cooper, http://dbpedia.org/resource/Brian_Kerwin, http://dbpedia.org/resource/Claude_Akins, http://dbpedia.org/resource/Mills_Watson does not look li

nb : 968
Subject: http://dbpedia.org/resource/The_Kingdom_(miniseries), Predicate: http://dbpedia.org/ontology/starring, Object: http://dbpedia.org/resource/Birgitte_Raaberg, http://dbpedia.org/resource/Holger_Juul_Hansen, http://dbpedia.org/resource/Peter_Mygind, http://dbpedia.org/resource/Udo_Kier, http://dbpedia.org/resource/Ghita_Nørby, http://dbpedia.org/resource/Annevig_Schelde_Ebbe, http://dbpedia.org/resource/John_Hahn-Petersen, http://dbpedia.org/resource/Erik_Wedersøe, http://dbpedia.org/resource/Ernst-Hugo_Järegård, http://dbpedia.org/resource/Otto_Brandenburg, http://dbpedia.org/resource/Baard_Owe, http://dbpedia.org/resource/Solbjørg_Højfeldt, http://dbpedia.org/resource/Jens_Okking, http://dbpedia.org/resource/Birthe_Neumann, http://dbpedia.org/resource/Søren_Pilmark, http://dbpedia.org/resource/Kirsten_Rolffes, http://dbpedia.org/resource/Vita_Jensen
Subject: http://dbpedia.org/resource/Lizzie_Borden_Took_an_Ax, Predicate: http://dbpedia.org/ontology/runtime, Object: 52

## 3. Analyse exploratoire des données 

- Créer des graphiques permettant de visualiser la distribution des films et séries dans votre base de données.
- Créer un graphique pour montrer les films et les séries les mieux notés
- Créer un nuage de points pour représenter la relation entre les caractéristiques

## 4. Système de Recommandation 

a) Utiliser SPARQL pour interroger le graphe RDF afin de créer un système de recommandation de films et/ou séries. Vous pouvez envisager différentes approches de recommandation, telles que la recommandation collaborative (en fonction des évaluations d'utilisateurs similaires) ou la recommandation basée sur le contenu (en fonction des genres, des acteurs, etc.) ou la recommandation basée sur les connaissances

b) Utiliser une IA (ChatGPT ou tout autre) pour répondre à cette question

## 5. Calcul des Recommandations 

a) Utiliser SPARQL pour générer des requêtes de recommandation en fonction des préférences de l'utilisateur. Vous pouvez également utiliser des algorithmes d'apprentissage automatique pour améliorer les recommandations.

b) Utiliser une IA pour répondre à cette question

## 6. Évaluation  

a) Évaluez la qualité de vos recommandations en utilisant des mesures telles que  MAP@k (Mean Average Precision at k) ou NDCG@k (Normalized Discounted Cumulative Gain at k)

b) Comparer les recommandations que vous avez obtenues par rapport à celles générées par l'IA que vous aurez utilisée

## 7. Rapport 

Vous allez rédiger un rapport de 5 pages max décrivant la modélisation RDF, les requêtes SPARQL, l'algorithme de recommandation et les résultats de l'évaluation. Vous spécifierez votre utilisation de l'IA dans le cadre de ce projet et ce que vous en pensez